## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Wanning,18.8003,110.3967,70.00,90,99,5.84,CN,overcast clouds
1,1,Tura,25.5198,90.2201,52.05,81,18,2.75,IN,few clouds
2,2,Tiksi,71.6872,128.8694,-15.12,98,100,2.91,RU,overcast clouds
3,3,Presidencia Roque Saenz Pena,-26.7852,-60.4388,85.60,28,14,8.61,AR,few clouds
4,4,Vaini,-21.2000,-175.2000,75.36,94,100,12.66,TO,light rain


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature for your trip?"))
max_temp = float(input("What is the maximum temperature for your trip?"))

What is the minimum temperature for your trip?75
What is the maximum temperature for your trip?90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & (city_data_df["Max Temp"]>= min_temp)]
clean_df.head()   

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
3,3,Presidencia Roque Saenz Pena,-26.7852,-60.4388,85.60,28,14,8.61,AR,few clouds
4,4,Vaini,-21.2000,-175.2000,75.36,94,100,12.66,TO,light rain
11,11,Hithadhoo,-0.6000,73.0833,82.69,72,21,12.66,MV,few clouds
14,14,Caravelas,-17.7125,-39.2481,84.85,60,4,10.60,BR,clear sky
15,15,Nieuw Amsterdam,5.8833,-55.0833,87.82,69,40,5.75,SR,scattered clouds


In [15]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID        219
City           219
Lat            219
Lng            219
Max Temp       219
Humidity       219
Cloudiness     219
Wind Speed     219
Country        219
Description    219
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Not needed!

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Presidencia Roque Saenz Pena,AR,85.60,few clouds,-26.7852,-60.4388,
4,Vaini,TO,75.36,light rain,-21.2000,-175.2000,
11,Hithadhoo,MV,82.69,few clouds,-0.6000,73.0833,
14,Caravelas,BR,84.85,clear sky,-17.7125,-39.2481,
15,Nieuw Amsterdam,SR,87.82,scattered clouds,5.8833,-55.0833,
18,Castro,BR,77.77,broken clouds,-24.7911,-50.0119,
26,Port Alfred,ZA,75.65,clear sky,-33.5906,26.8910,
27,Rikitea,PF,79.02,broken clouds,-23.1203,-134.9692,
29,Atuona,PF,78.75,clear sky,-9.8000,-139.0333,
31,Mahibadhoo,MV,82.02,few clouds,3.7833,72.9667,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City           219
Country        219
Max Temp       219
Description    219
Lat            219
Lng            219
Hotel Name     219
dtype: int64

In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))